In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
H, W, C = 64, 64, 3
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [ ]:
def load_images(path):
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image)
    image = tf.image.resize(image, (H, W))
    image = tf.cast(image, tf.float32)
    image = (image - 127.5)/ 127.5
    return image

In [ ]:
def tf_dataset(images_path, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(images_path)
    dataset = dataset.shuffle(buffer_size=1024)
    dataset = dataset.map(load_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
def deconv_block(inputs, num_filters, kernel_size, strides, batch_norm = True):
    x = Conv2DTranspose(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding="same",
        strides=strides,
        use_bias=False,
        
    )(inputs)

    if batch_norm:
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)

    return x

In [ ]:
def conv_block(inputs, num_filters, kernel_size, padding="same", strides=2, activation=True):
    x = Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding=padding,
        strides=strides,
        
    )(inputs)

    if activation:
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.3)(x)

    return x

In [ ]:
def build_generator(latent_dim):
    f = [2 ** i for i in range(5)][::-1]
    filters = 32
    output_strides = 16
    h_output = H // output_strides
    w_output = W // output_strides

    noise = Input(shape=(latent_dim,), name="Gen_noise_input")

    x = Dense(f[0] * filters * h_output * w_output, use_bias=False)(noise)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((h_output, w_output, f[0] * filters))(x)

    for i in range(1, 5):
        x = deconv_block(x,
                    num_filters=f[i] * filters,
                    kernel_size=5,
                    strides=2,
                    batch_norm=True)
        #print(x.shape)
    x = conv_block(x,
                  num_filters=3,
                  kernel_size=5,
                  strides=1,
                  activation=False)
    fake_output = Activation("tanh")(x)

    return Model(noise, fake_output, name="generator")

In [ ]:
def build_discriminator():
    f = [2 ** i for i in range(4)]
    filters = 64
    output_strides = 16
    h_output = H // output_strides
    w_output = W // output_strides

    image_input = Input(shape=(H, W, C), name="images")
    x = image_input
    for i in range(0, 4):
        x = conv_block(x, 
                      num_filters=f[i] * filters,
                      kernel_size=5,
                      strides=2)

    x = Flatten()(x)
    x = Dense(1)(x)

    return Model(image_input, x, name="Discriminator")

In [ ]:
class GAN(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]

        """ Training Discriminator """
        for _ in range(2):
            # Fake Images
            random_latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
            generator_iamges = self.generator(random_latent_vector)
            generator_labels = tf.zeros((batch_size, 1))

            with tf.GradientTape() as ftape:
                prediction = self.discriminator(generator_iamges)
                d1_loss = self.loss_fn(generator_labels, prediction)
            grads = ftape.gradient(d1_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

            # Real Images
            labels = tf.ones((batch_size, 1))
            with tf.GradientTape() as rtape:
                prediction = self.discriminator(real_images)
                d2_loss = self.loss_fn(labels, prediction)
            grads = rtape.gradient(d2_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
            

        """ Training Generator """
        random_latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.ones((batch_size, 1))
        
        with tf.GradientTape() as gtape:
            prediction = self.discriminator(self.generator(random_latent_vector))
            g_loss = self.loss_fn(misleading_labels, prediction)
        grads = gtape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d1_loss":d1_loss, "d2_loss":d2_loss, "g_loss":g_loss}

In [ ]:
def save_plot(examples, epoch, n):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        plt.subplot(n, n, i + 1)
        plt.axis("off")
        plt.imshow(examples[i])
    file_name = f"fake_image_epoch_{epoch + 1}.png"
    plt.savefig(file_name)
    plt.close()

In [ ]:
if __name__ == "__main__":

    """ Hyperparameters """
    batch_size = 128
    latent_dim = 128
    num_epochs = 100

    data_path = "data/data/*"
    images_path = glob(data_path)
    #print(f"Dataset Size : {len(images_path)}")

    d_model = build_discriminator()
    g_model = build_generator(latent_dim)

    #d_model.summary()
    #g_model.summary()

    #print(f"Build_generator : {build_generator(latent_dim)}")

    # dataset = tf_dataset(images_path, batch_size=batch_size)
    # for x in dataset:
    #     print(x.shape)
    #     break

    gan = GAN(d_model, g_model, latent_dim)

    bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
    d_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    g_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

    gan.compile(d_optimizer, g_optimizer, bce_loss_fn)

    images_dataset = tf_dataset(images_path, batch_size)
    for epoch in range(num_epochs):

        gan.fit(images_dataset, epochs=1)

        g_model.save("Generative_models/G_model.h5")
        d_model.save("Generative_models/D_model.h5")
    
        n_samples = 25
        noise = np.random.normal(size=(n_samples, latent_dim))
        examples = g_model.predict(noise)
        save_plot(examples, epoch, int(np.sqrt(n_samples)))
        #gan.summary()

In [ ]:
g_model = load_model("Generative_modelsG_model.h5")
n_samples = 25
latent_dim = 128
noise = np.random.normal(size=(n_samples, latent_dim))
examples = g_model.predict(noise)
save_plot(examples, epoch, int(np.sqrt(n_samples)))

In [ ]:
def save_plot_1(examples, n):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        plt.subplot(n, n, i + 1)
        plt.axis("off")
        plt.imshow(examples[i])
    file_name = f"Generative_models/fake_image.png"
    plt.savefig(file_name)
    plt.close()

In [ ]:
if __name__ == "__main__":
    model = load_model(r"E:\python\segmentation\Computer Vision\UNET\Gan\Generative_models\G_model.h5")

    n_samples = 25
    latent_dim = 128
    latent_points = np.random.normal(size=(n_samples, latent_dim))
    examples = model.predict(latent_points)
    save_plot_1(examples, 5)